<a href="https://colab.research.google.com/github/Aatiqa-bano/Internship-Projects/blob/main/Laptop_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#libraries used

In [103]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [104]:
df = pd.read_csv('laptopPrice.csv')

In [105]:
print(df.head())

    brand processor_brand processor_name processor_gnrtn  ram_gb ram_type  \
0    ASUS           Intel        Core i3            10th       4     DDR4   
1  Lenovo           Intel        Core i3            10th       4     DDR4   
2  Lenovo           Intel        Core i3            10th       4     DDR4   
3    ASUS           Intel        Core i5            10th       8     DDR4   
4    ASUS           Intel   Celeron Dual   Not Available       4     DDR4   

   ssd   hdd       os  os_bit  graphic_card_gb  weight     warranty  \
0    0  1024  Windows      64                0  Casual  No warranty   
1    0  1024  Windows      64                0  Casual  No warranty   
2    0  1024  Windows      64                0  Casual  No warranty   
3  512     0  Windows      32                2  Casual  No warranty   
4    0   512  Windows      64                0  Casual  No warranty   

  Touchscreen msoffice  Price  rating  Number of Ratings  Number of Reviews  
0          No       No  34649   

# Summary statistics


In [106]:
print(df.describe())

           ram_gb          ssd          hdd      os_bit  graphic_card_gb  \
count  802.000000   802.000000   802.000000  802.000000       802.000000   
mean     8.673317   453.745636   231.102244   59.650873         1.216958   
std      4.388785   318.931839   415.928535   10.973026         2.042082   
min      4.000000     0.000000     0.000000   32.000000         0.000000   
25%      4.000000   256.000000     0.000000   64.000000         0.000000   
50%      8.000000   512.000000     0.000000   64.000000         0.000000   
75%      8.000000   512.000000     0.000000   64.000000         2.000000   
max     32.000000  3072.000000  2048.000000   64.000000         8.000000   

               Price      rating  Number of Ratings  Number of Reviews  
count     802.000000  802.000000          802.00000         802.000000  
mean    76625.543641    3.568579          299.84414          36.089776  
std     45232.984422    0.568337         1001.78442         118.313553  
min     16990.000000   

# Check for missing values


In [107]:
print(df.isnull().sum())

brand                0
processor_brand      0
processor_name       0
processor_gnrtn      0
ram_gb               0
ram_type             0
ssd                  0
hdd                  0
os                   0
os_bit               0
graphic_card_gb      0
weight               0
warranty             0
Touchscreen          0
msoffice             0
Price                0
rating               0
Number of Ratings    0
Number of Reviews    0
dtype: int64


# Feature Engineering


In [108]:
# Combine SSD and HDD into a single feature for total storage
df['total_storage'] = df['ssd'] + df['hdd']

In [109]:
# Create a new feature: Price per GB of storage
df['price_per_gb'] = df['Price'] / (df['total_storage'] + 1)  # Adding 1 to avoid division by zero

In [110]:
# Convert processor generation to numerical values (e.g., 10th generation as 10)
def process_generation(gen):
    try:
        return int(gen.split(' ')[0])
    except ValueError:
        return 0  # Default value for missing or non-numeric generations

# Check if 'processor_gnrtn' exists before applying the function
if 'processor_gnrtn' in df.columns:
    df['processor_gnrtn_num'] = df['processor_gnrtn'].apply(process_generation)

# Drop columns no longer needed
df.drop(['processor_gnrtn', 'total_storage'], axis=1, inplace=True, errors='ignore')

# Convert categorical features to numerical values using one-hot encoding
df_encoded = pd.get_dummies(df, drop_first=True)

# Check the new DataFrame
print(df_encoded.head())

# Prepare features and target variable
X = df_encoded.drop('Price', axis=1, errors='ignore')  # Use errors='ignore' to avoid KeyError if 'Price' is missing
y = df_encoded['Price']

   ram_gb  ssd   hdd  os_bit  graphic_card_gb  Price  rating  \
0       4    0  1024      64                0  34649       2   
1       4    0  1024      64                0  38999       3   
2       4    0  1024      64                0  39999       3   
3       8  512     0      32                2  69990       3   
4       4    0   512      64                0  26990       3   

   Number of Ratings  Number of Reviews  price_per_gb  ...  ram_type_LPDDR4X  \
0                  3                  0     33.803902  ...             False   
1                 65                  5     38.047805  ...             False   
2                  8                  1     39.023415  ...             False   
3                  0                  0    136.432749  ...             False   
4                  0                  0     52.612086  ...             False   

   os_Mac  os_Windows  weight_Gaming  weight_ThinNlight  warranty_2 years  \
0   False        True          False              False  

# Train-test split


In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model


In [112]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

# Predict and evaluate


In [113]:
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

MAE: 9512.199682877359
RMSE: 15165.386405123754
